In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint


from api_keys import g_key


In [2]:
output_data_file = "output_data/cities.csv"

weather_df = pd.read_csv(output_data_file)

weather_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_df[["Lat", "Lng"]]

humidity = weather_df["Humidity"].astype(float)


fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=10,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
new_df = pd.DataFrame(weather_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_df["Max Temp"] <= 80) & (new_df["Max Temp"] > 70)
wind_speed = new_df["Wind Speed"] < 10
cloudiness = new_df["Cloudiness"] == 0

new_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
7,opuwo,72.06,5.44,0
70,ponta do sol,73.68,5.32,0
88,cidreira,76.56,5.50,0
138,morondava,79.08,9.06,0
139,vaini,78.90,1.59,0
176,nador,75.20,6.93,0
297,mogok,72.24,2.77,0
444,birjand,71.60,9.17,0


In [6]:
hotel= pd.DataFrame(weather_df, columns=["City", "Country", "Lat", "Lng"])

hotel["Hotel Name"] = ""
hotel

,City,Country,Lat,Lng,Hotel Name
0,ostrovnoy,RU,68.05,39.51,
1,mahebourg,MU,-20.41,57.70,
2,qaanaaq,GL,77.48,-69.36,
3,zhuhai,CN,40.71,112.04,
4,cape town,ZA,-33.93,18.42,
...,...,...,...,...,...
543,yabrud,SY,33.97,36.66,
544,paraiso,MX,24.01,-104.61,
545,veraval,FR,49.65,0.71,
546,novyy urgal,RU,51.07,132.56,


In [7]:
coordinates_lat_lng = []

city = hotel["City"]
city

0        ostrovnoy
1        mahebourg
2          qaanaaq
3           zhuhai
4        cape town
          ...     
543         yabrud
544        paraiso
545        veraval
546    novyy urgal
547        tongren
Name: City, Length: 548, dtype: object

In [8]:
coordinates = f"{hotel['Lat'][0]},{hotel['Lng'][0]}"
search = "hotel"
radius = 5000
type = "hotel"

params = {
    "location": coordinates,
    "keyword": search,
    "radius": radius,
    "type": type,
    "key": g_key}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

response = requests.get(base_url, params=params)
pprint(response.json(), depth=1)


{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}


In [9]:
hotel.iloc[0][0]

'ostrovnoy'

In [10]:
hotel_name = []

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    try:
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)
        
hotel_name

ConnectionError: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))

In [ ]:
hotel['Hotel Name']= hotel_name
hotel.dropna()
hotel.to_csv('hotel_export.csv')

In [ ]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel.iterrows()]
locations = hotel[["Lat", "Lng"]]

In [ ]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig